In [1]:
import json
from pathlib import Path
import numpy as np


def print_transition_matrix(transition_matrix):
    print("    | " + " ".join([f"   {i+1}   " for i in range(4)]))
    print("----+" + "--------" * 4)
    for i in range(3):
        print(
            f" {i+1}  | "
            + "  ".join([f"{transition_matrix[i,j]:.4f}" for j in range(4)])
        )


result_file = Path(f"button_usage.jsonl")
if result_file.exists():
    data = list(map(lambda x: json.loads(x), open(result_file).readlines()))

    print(f"The number of collections: {len(data)}")

    learn_costs = np.array([x["learn_costs"] for x in data])
    review_costs = np.array([x["review_costs"] for x in data])

    learn_costs_mean = np.median(learn_costs, axis=0).round(3).tolist()
    review_costs_mean = np.median(review_costs, axis=0).round(3).tolist()
    print(f"Learn costs median: {learn_costs_mean}")
    print(f"Review costs median: {review_costs_mean}")

    first_rating_probs = np.array([x["first_rating_prob"] for x in data])
    review_rating_probs = np.array([x["review_rating_prob"] for x in data])

    first_rating_probs_mean = np.mean(first_rating_probs, axis=0).round(3).tolist()
    review_rating_probs_mean = np.nanmean(review_rating_probs, axis=0).round(3).tolist()
    print(f"First rating prob mean: {first_rating_probs_mean}")
    print(f"Review rating prob mean: {review_rating_probs_mean}")

    first_rating_offsets = np.array([x["first_rating_offset"] for x in data])
    first_session_lens = np.array([x["first_session_len"] for x in data])

    first_rating_offsets_median = (
        np.median(first_rating_offsets, axis=0).round(3).tolist()
    )
    first_session_lens_median = np.median(first_session_lens, axis=0).round(3).tolist()
    print(f"First rating offset median: {first_rating_offsets_median}")
    print(f"First session len median: {first_session_lens_median}")

    forget_rating_offsets = np.array([x["forget_rating_offset"] for x in data])
    forget_session_lens = np.array([x["forget_session_len"] for x in data])

    forget_rating_offsets_median = np.median(forget_rating_offsets).round(3)
    forget_session_lens_median = np.median(forget_session_lens).round(3)
    print(f"Forget rating offset median: {forget_rating_offsets_median}")
    print(f"Forget session len median: {forget_session_lens_median}")

    short_term_recall = np.array([x["short_term_recall"] for x in data])
    short_term_recall = short_term_recall[
        ~(short_term_recall == 0).any(axis=1) & ~(short_term_recall == 1).any(axis=1)
    ]
    short_term_recall_mean = np.mean(short_term_recall, axis=0).round(3).tolist()
    print(f"Short term recall mean: {short_term_recall_mean}")

    learning_step_transition = np.array([x["learning_step_transition"] for x in data])
    learning_step_transition_total = np.sum(learning_step_transition, axis=0).tolist()
    transition_matrix = np.zeros((3, 4))
    for i in range(len(learning_step_transition_total)):
        row_sum = np.sum(learning_step_transition_total[i])
        transition_matrix[i] = learning_step_transition_total[i] / row_sum
    print(f"Learning step transition:")
    print_transition_matrix(transition_matrix)

    relearning_step_transition = np.array(
        [x["relearning_step_transition"] for x in data]
    )
    relearning_step_transition_total = np.sum(
        relearning_step_transition, axis=0
    ).tolist()
    transition_matrix = np.zeros((3, 4))
    for i in range(len(relearning_step_transition_total)):
        row_sum = np.sum(relearning_step_transition_total[i])
        transition_matrix[i] = relearning_step_transition_total[i] / row_sum
    print(f"Relearning step transition:")
    print_transition_matrix(transition_matrix)

    state_rating_costs = np.array([x["state_rating_costs"] for x in data])
    state_rating_costs_median = np.median(state_rating_costs, axis=0).round(2).tolist()
    print(f"State rating costs median: {state_rating_costs_median}")

    print(
        f"learn cost: {np.median(np.sum(learn_costs * first_rating_probs, axis=1)).round(4)}"
    )
    print(
        f"pass cost: {np.nanmedian(np.sum(review_costs[:, 1:] * review_rating_probs, axis=1)).round(4)}"
    )
    print(f"fail cost: {np.median(review_costs[:, 0]).round(4)}")
    print(f"initial pass prob: {np.mean(1 - first_rating_probs[:, 0]).round(4)}")

The number of collections: 10000
Learn costs median: [34.545, 27.0, 14.16, 6.78]
Review costs median: [23.185, 11.81, 7.48, 5.64]
First rating prob mean: [0.236, 0.103, 0.489, 0.173]
Review rating prob mean: [0.231, 0.626, 0.143]
First rating offset median: [-0.75, -0.19, -0.01, 0.0]
First session len median: [2.04, 1.43, 0.82, 0.0]
Forget rating offset median: -0.27
Forget session len median: 1.06
Short term recall mean: [0.742, 0.917, 0.962, 0.853]
Learning step transition:
    |    1       2       3       4   
----+--------------------------------
 1  | 0.3686  0.0628  0.5108  0.0577
 2  | 0.0442  0.4553  0.4457  0.0549
 3  | 0.0519  0.0470  0.8462  0.0550
Relearning step transition:
    |    1       2       3       4   
----+--------------------------------
 1  | 0.2157  0.0643  0.6595  0.0604
 2  | 0.0500  0.4638  0.4475  0.0387
 3  | 0.1057  0.1434  0.7266  0.0244
State rating costs median: [[12.75, 12.26, 8.0, 6.38], [13.05, 11.74, 7.42, 5.6], [10.56, 10.0, 7.37, 5.4]]
learn cos